# <a id='supervised-learning'></a> 1. [**Apprentissage supervisé**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_supervised_learning.ipynb#supervised-learning)</br>([*Supervised learning*](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning))

# <a id='probability-calibration'></a> 1.16. [**Étalonnage de probabilité**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_16_calibration.ipynb)<br/>([_Probability calibration_](https://scikit-learn.org/stable/modules/calibration.html))

# Sommaire

- **Volume** : 6 pages, 4 exemples, 8 papiers
- 1.16.1. [**Courbes d'étalonnage**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_16_calibration.ipynb#calibration-curves)<br/>([_Calibration curves_](https://scikit-learn.org/stable/modules/calibration.html#calibration-curves))
- 1.16.2. [**Étalonnage d'un classifieur**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_16_calibration.ipynb#calibrating-a-classifier)<br/>([_Calibrating a classifier_](https://scikit-learn.org/stable/modules/calibration.html#calibrating-a-classifier))
- 1.16.3. [**Utilisation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_16_calibration.ipynb#usage)<br/>([_Usage_](https://scikit-learn.org/stable/modules/calibration.html#usage))
    - 1.16.3.1. [**Sigmoïde**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_16_calibration.ipynb#sigmoid)<br/>([_Sigmoid_](https://scikit-learn.org/stable/modules/calibration.html#isotonic))
    - 1.16.3.2. [**Isotone**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_16_calibration.ipynb#sigmoid)<br/>([_Isotonic_](https://scikit-learn.org/stable/modules/calibration.html#isotonic))
    - 1.16.3.3. [**Support multi-classes**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/1_16_calibration.ipynb#multiclass-support)<br/>([_Multiclass support_](https://scikit-learn.org/stable/modules/calibration.html#multiclass-support))

# <a id='probability-calibration'></a> 1.16. **Étalonnage de probabilité**<br/>([_Probability calibration_](https://scikit-learn.org/stable/modules/calibration.html))

Lorsque vous effectuez une classification, vous souhaitez souvent non seulement prédire l'étiquette de classe, mais aussi obtenir une probabilité de l'étiquette respective. Cette probabilité vous donne une sorte de confiance dans la prédiction. Certains modèles peuvent vous donner de mauvaises estimations des probabilités de classe, et certains ne prennent même pas en charge la prédiction de probabilité (par exemple, certaines instances de [**`SGDClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)). Le module d'étalonnage vous permet d'étalonner plus finement les probabilités d'un modèle donné ou d'ajouter la prise en charge de la prédiction de probabilité.

Les classifieurs bien étalonnés sont des classifieurs probabilistes pour lesquels la sortie de la méthode [**`predict_proba`**](https://scikit-learn.org/stable/glossary.html#term-predict_proba) peut être directement interprétée comme un niveau de confiance. Par exemple, un classifieur binaire bien étalonné devrait classer les échantillons de telle sorte que parmi les échantillons auxquels il a attribué une valeur de `predict_proba` proche, disons, de 0,8, environ 80 % appartiennent réellement à la classe positive.

Avant de montrer comment ré-étalonner un classifieur, nous avons d'abord besoin d'une méthode pour détecter à quel point un classifieur est bien étalonné.

> **Remarque :** Les règles d'évaluation strictement appropriées pour les prédictions probabilistes, telles que [**`sklearn.metrics.brier_score_loss`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html#sklearn.metrics.brier_score_loss) et [**`sklearn.metrics.log_loss`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss), évaluent simultanément l'étalonnage (la fiabilité) et la puissance discriminatoire (la résolution) d'un modèle, ainsi que le caractère aléatoire des données (l'incertitude). Cela découle de la fameuse décomposition de Murphy du score de Brier [1]. Comme le terme dominant n'est pas clairement identifiable, le score est d'une utilité limitée pour évaluer l'étalonnage seul (à moins que l'on ne calcule chaque terme de la décomposition). Une perte de Brier plus faible, par exemple, ne signifie pas nécessairement un modèle mieux étalonné, et pourrait même signifier un modèle moins bien étalonné avec beaucoup plus de puissance discriminatoire, par exemple en utilisant de nombreuses autres caractéristiques.

## <a id='calibration-curves'></a> 1.16.1. **Courbes d'étalonnage**<br/>([_Calibration curves_](https://scikit-learn.org/stable/modules/calibration.html#calibration-curves))

Les courbes d'étalonnage, également appelées _diagrammes de fiabilité_ (Wilks 1995 [2]), permettent de comparer la qualité de l'étalonnage des prédictions probabilistes d'un classifieur binaire. Elles tracent la fréquence de l'étiquette positive (plus précisément, une estimation de la _probabilité conditionnelle d'un événement_ $P(Y=1|\text{predict\_proba})$) sur l'axe des ordonnées par rapport à la probabilité prédite [**`predict_proba`**](https://scikit-learn.org/stable/glossary.html#term-predict_proba) d'un modèle sur l'axe des abscisses. La partie délicate consiste à obtenir des valeurs pour l'axe des ordonnées. Dans scikit-learn, cela est accompli en regroupant les prédictions de manière à ce que l'axe des abscisses représente la probabilité prédite moyenne dans chaque groupe. L'axe des ordonnées est alors la _fraction de positifs_ étant donné les prédictions de ce groupe, c'est-à-dire la proportion d'échantillons dont la classe est la classe positive (dans chaque groupe).

Le graphique de la courbe d'étalonnage supérieure est créé avec [**`CalibrationDisplay.from_estimator`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibrationDisplay.html#sklearn.calibration.CalibrationDisplay.from_estimator), qui utilise [**`calibration_curve`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.calibration_curve.html#sklearn.calibration.calibration_curve) pour calculer les probabilités moyennes prédites par groupe et la fraction de positifs. [**`CalibrationDisplay.from_estimator`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibrationDisplay.html#sklearn.calibration.CalibrationDisplay.from_estimator) prend en entrée un classifieur ajusté, qui est utilisé pour calculer les probabilités prédites. Le classifieur doit donc avoir une méthode `predict_proba`. Pour les rares classifieurs qui n'ont pas de méthode [**predict_proba**](https://scikit-learn.org/stable/glossary.html#term-predict_proba), il est possible d'utiliser [**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV) pour étalonner les sorties du classifieur en probabilités.

L'histogramme inférieur donne un aperçu du comportement de chaque classifieur en montrant le nombre d'échantillons dans chaque groupe de probabilités prédites.

<div style="background-color: white; color: black; text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/sphx_glr_plot_compare_calibration_001.png"
    alt="Courbes d'étalonnage"
    style="max-width: 75%; height; auto;"/>
</div>

[**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression) retourne par défaut des prédictions bien étalonnées, car il possède une fonction de liaison canonique pour sa perte, c'est-à-dire la fonction de liaison logit pour la [**Perte logarithmique** (3.3.2.12)](https://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Cela conduit à ce que l'on appelle la **propriété d'équilibre**, voir [8] et [**Régression logistique** (1.1.11)](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression). En revanche, les autres modèles présentés renvoient des probabilités biaisées, avec des biais différents pour chaque modèle.

[**`GaussianNB`**](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB) (Naive Bayes) a tendance à pousser les probabilités vers 0 ou 1 (notez les décomptes dans les histogrammes). Cela est principalement dû au fait qu'il suppose que les caractéristiques sont conditionnellement indépendantes étant donné la classe, ce qui n'est pas le cas dans cet ensemble de données qui contient 2 caractéristiques redondantes.

[**`RandomForestClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) montre un comportement opposé : les histogrammes montrent des pics de probabilités d'environ 0,2 et 0,9, tandis que les probabilités proches de 0 ou 1 sont très rares. Niculescu-Mizil et Caruana [3] en donnent une interprétation : _"Les méthodes telles que le bagging et les forêts aléatoires qui moyennent les prédictions à partir d'un ensemble de modèles de base peuvent avoir des difficultés à faire des prédictions près de 0 et 1, car la variance des modèles de base sous-jacents biaisera les prédictions qui devraient être près de zéro ou de un, les éloignant de ces valeurs. Étant donné que les prédictions sont limitées à l'intervalle [0,1], les erreurs causées par la variance ont tendance à être unilatérales près de zéro et de un. Par exemple, si un modèle devrait prédire p = 0 pour un cas, la seule façon pour le bagging d'y parvenir est que tous les arbres mis en sac prédisent zéro. Si nous ajoutons du bruit aux arbres sur lesquels repose le bagging, ce bruit fera que certains arbres prédisent des valeurs supérieures à 0 pour ce cas, déplaçant ainsi la prédiction moyenne de l'ensemble mis en sac loin de zéro. Nous observons cet effet de manière plus marquée avec les forêts aléatoires car les arbres de niveau de base formés avec les forêts aléatoires ont une variance relativement élevée en raison de la sélection de caractéristiques."_ En conséquence, la courbe d'étalonnage montre une forme sigmoïde caractéristique, indiquant que le classifieur pourrait avoir davantage confiance en son "intuition" et renvoyer des probabilités généralement plus proches de 0 ou 1.

[**`LinearSVC`**](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) (SVC) montre une courbe encore plus sigmoïde que la forêt aléatoire, ce qui est typique des méthodes à marge maximale (comparez avec Niculescu-Mizil et Caruana [3]), qui se concentrent sur les échantillons difficiles à classer qui sont proches de la frontière de décision (les vecteurs de support).

## <a id='calibrating-a-classifier'></a> 1.16.2. **Étalonnage d'un classifieur**<br/>([_Calibrating a classifier_](https://scikit-learn.org/stable/modules/calibration.html#calibrating-a-classifier))

L'étalonnage d'un classifieur consiste à ajuster un régresseur (appelé étalonneur) qui fait correspondre la sortie du classifieur (telle qu'elle est fournie par la méthode [**`decision_function`**](https://scikit-learn.org/stable/glossary.html#term-decision_function) ou [**`predict_proba`**](https://scikit-learn.org/stable/glossary.html#term-predict_proba)) à une probabilité étalonnée dans l'intervalle $[0, 1]$. En notant la sortie du classifieur pour un échantillon donné par $f_i$, l'étalonneur tente de prédire la probabilité conditionnelle de l'événement $P(y_i = 1 | f_i)$.

Idéalement, l'étalonneur est ajusté sur un ensemble de données indépendant de l'ensemble d'entraînement utilisé pour ajuster le classifieur en premier lieu. Cela est dû au fait que les performances du classifieur sur ses données d'entraînement seraient meilleures que pour de nouvelles données. Utiliser la sortie du classifieur des données d'entraînement pour ajuster l'étalonneur aboutirait ainsi à un étalonneur biaisé qui renverrait des probabilités plus proches de 0 et 1 qu'il ne le devrait.

## <a id='usage'></a> 1.16.3. **Utilisation**<br/>([_Usage_](https://scikit-learn.org/stable/modules/calibration.html#usage))

La classe [**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV) est utilisée pour étalonner un classifieur.

[**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV) utilise une approche de validation croisée pour garantir que des données non biaisées sont toujours utilisées pour ajuster l'étalonneur. Les données sont divisées en $k$ couples `(train_set, test_set)` (tels que déterminés par `cv`). Lorsque `ensemble=True` (par défaut), la procédure suivante est répétée indépendamment pour chaque division de validation croisée : un clone de `base_estimator` est d'abord entraîné sur le sous-ensemble d'entraînement. Ensuite, ses prédictions sur le sous-ensemble de test sont utilisées pour ajuster un étalonneur (un régresseur sigmoïde ou isotone). Cela résulte en un ensemble de $k$ couples `(classifieur, étalonneur)` où chaque étalonneur fait correspondre la sortie de son classifieur correspondant dans l'intervalle $[0, 1]$. Chaque couple est exposé dans l'attribut `calibrated_classifiers_`, où chaque entrée est un classifieur étalonné avec une méthode [**`predict_proba`**](https://scikit-learn.org/stable/glossary.html#term-predict_proba) qui renvoie des probabilités étalonnées. La sortie de `predict_proba` pour l'instance principale de [**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV) correspond à la moyenne des probabilités prédites des $k$ estimateurs de la liste `calibrated_classifiers_`. La sortie de `predict` est la classe ayant la probabilité la plus élevée.

Lorsque `ensemble=False`, la validation croisée est utilisée pour obtenir des prédictions "non biaisées" pour l'ensemble des données, via [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html#sklearn.model_selection.cross_val_predict). Ces prédictions non biaisées sont ensuite utilisées pour entraîner l'étalonneur. L'attribut `calibrated_classifiers_` se compose uniquement d'un couple `(classifieur, étalonneur)` où le classifieur est le `base_estimator` entraîné sur l'ensemble des données. Dans ce cas, la sortie de [**`predict_proba`**](https://scikit-learn.org/stable/glossary.html#term-predict_proba) pour [**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV) correspond aux probabilités prédites obtenues à partir de l'unique couple `(classifieur, étalonneur)`.

Le principal avantage de `ensemble=True` est de bénéficier de l'effet d'ensemble traditionnel (similaire à [**Bagging meta-estimator** (1.11.3)](https://scikit-learn.org/stable/modules/ensemble.html#bagging)). L'ensemble résultant devrait à la fois être bien étalonné et légèrement plus précis qu'avec `ensemble=False`. Le principal avantage de l'utilisation de `ensemble=False` est d'ordre calculatoire : cela réduit le temps d'ajustement global en n'entraînant qu'une seule paire de classifieur de base et d'étalonneur, réduit la taille du modèle final et augmente la vitesse de prédiction.

En outre, un classifieur déjà ajusté peut être étalonné en définissant `cv="prefit"`. Dans ce cas, les données ne sont pas divisées et l'ensemble des données est utilisé pour ajuster le régresseur. Il revient à l'utilisateur de s'assurer que les données utilisées pour l'ajustement du classifieur sont disjointes des données utilisées pour l'ajustement du régresseur.

[**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV) prend en charge l'utilisation de deux techniques de régression pour l'étalonnage via le paramètre `method` : `"sigmoid"` et `"isotonic"`.

### <a id='sigmoid'></a> 1.16.3.1. **Sigmoïde**<br/>([_Sigmoid_](https://scikit-learn.org/stable/modules/calibration.html#isotonic))

Le régresseur sigmoïde, `method="sigmoid"`, est basé sur le modèle logistique de Platt [4] :

$$p(y_i = 1 | f_i) = \frac{1}{1 + \exp(A f_i + B)} \,,$$

où $y_i$ est la véritable étiquette de l'échantillon $i$ et $f_i$ est la sortie du classifieur non étalonné pour l'échantillon $i$. Les valeurs réelles de $A$ et $B$ sont déterminées lors de l'ajustement du régresseur par maximum de vraisemblance.

La méthode sigmoïde suppose que la [**courbe d'étalonnage** (1.16.1)](https://scikit-learn.org/stable/modules/calibration.html#calibration-curve) peut être corrigée en appliquant une fonction sigmoïde aux prédictions brutes. Cette hypothèse a été justifiée empiriquement dans le cas des [**Machines à Vecteurs de Support** (1.4)](https://scikit-learn.org/stable/modules/svm.html#svm) avec des fonctions noyau courantes sur divers jeux de données de référence à la section 2.1 de Platt 1999 [4], mais cela ne s'applique pas nécessairement en général. De plus, le modèle logistique fonctionne mieux si l'erreur d'étalonnage est symétrique, c'est-à-dire que la sortie du classifieur pour chaque classe binaire suit une distribution normale avec la même variance [7]. Cela peut poser problème pour des problèmes de classification fortement déséquilibrés, où les sorties n'ont pas une variance égale.

En général, cette méthode est la plus efficace pour les petites tailles d'échantillons ou lorsque le modèle non étalonné manque de confiance et présente des erreurs d'étalonnage similaires pour des sorties élevées et faibles.

### <a id='isotonic'></a> 1.16.3.2. **Isotone**<br/>([_Isotonic_](https://scikit-learn.org/stable/modules/calibration.html#isotonic))


La méthode `method="isotonic"` ajuste un régresseur isotone non paramétrique, qui génère une fonction croissante par morceaux, consultez [**`sklearn.isotonic`**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.isotonic). Il minimise :

$$\sum_{i=1}^{n} (y_i - \hat{f}_i)^2$$

sous réserve que $\hat{f}_i \geq \hat{f}_j$ chaque fois que $f_i \geq f_j$. $y_i$ est la véritable étiquette de l'échantillon $i$ et $\hat{f}_i$ est la sortie du classifieur étalonné pour l'échantillon $i (c'est-à-dire la probabilité étalonnée). Cette méthode est plus générale que `"sigmoid"` car la seule restriction est que la fonction de correspondance soit monotone. Elle est donc plus puissante car elle peut corriger toute distorsion monotone du modèle non étalonné. Cependant, elle est plus sujette au surajustement, en particulier sur de petits ensembles de données [6].

Dans l'ensemble, `"isotonic"` donnera des performances aussi bonnes, voire meilleures, que `"sigmoid"` lorsqu'il y a suffisamment de données (plus de ~ 1000 échantillons) pour éviter le surajustement [3].

> **Remarque :** Impact sur les métriques de classement telles que l'AUC  
> En général, on s'attend à ce que l'étalonnage n'affecte pas les métriques de classement telles que l'ROC-AUC. Cependant, ces métriques peuvent différer après étalonnage lors de l'utilisation de `method="isotonic"` car la régression isotone introduit des égalités dans les probabilités prédites. Cela peut être interprété comme une incertitude des prédictions du modèle. Si vous souhaitez strictement conserver le classement et donc les scores AUC, utilisez `method="logistic"`, qui est une transformation strictement monotone qui préserve donc le classement.

### <a id='multiclass-support'></a> 1.16.3.3. **Support multi-classes**<br/>([_Multiclass support_](https://scikit-learn.org/stable/modules/calibration.html#multiclass-support))

Les régresseurs isotone et sigmoïde prennent en charge uniquement les données à une dimension (par exemple, la sortie d'une classification binaire), mais sont étendus pour la classification multi-classes si le `base_estimator` prend en charge les prédictions multi-classes. Pour les prédictions multi-classes, [**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV) effectue l'étalonnage pour chaque classe séparément dans le style [**`OneVsRestClassifier`** (1.12.1.2)](https://scikit-learn.org/stable/modules/multiclass.html#ovr-classification) [5]. Lors de la prédiction des probabilités, les probabilités étalonnées pour chaque classe sont prédites séparément. Comme ces probabilités ne sont pas nécessairement normalisées de manière à ce que leur somme soit égale à un, un post-traitement est effectué pour les normaliser.

## Exemples

### [**Courbes d'étalonnage de probabilité**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/examples/1_16_calibration/plot_calibration_curve.ipynb)<br/>([_Probability Calibration curves_](https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html))

### [**Étalonnage de probabilité pour une classification à 3 classes**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/examples/1_16_calibration/plot_calibration_multiclass.ipynb)<br/>([_Probability Calibration for 3-class classification_](https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_multiclass.html))

### [**Étalonnage des probabilités des classifieurs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/examples/1_16_calibration/plot_calibration.ipynb)<br/>([_Probability calibration of classifiers_](https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration.html))

### [**Comparaison de l'étalonnage de classifieurs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/examples/1_16_calibration/plot_compare_calibration.ipynb)<br/>([_Comparison of Calibration of Classifiers_](https://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html))

## Références

🔬 [1] Allan H. Murphy (1973). [**“A New Vector Partition of the Probability Score”**](https://journals.ametsoc.org/downloadpdf/journals/apme/12/4/1520-0450_1973_012_0595_anvpot_2_0_co_2.pdf) Journal of Applied Meteorology and Climatology

🔬 [2] [**“On the combination of forecast probabilities for consecutive precipitation periods”**](https://journals.ametsoc.org/downloadpdf/journals/wefo/5/4/1520-0434_1990_005_0640_otcofp_2_0_co_2.xml). Wea. Forecasting, 5, 640–650., Wilks, D. S., 1990a

🔬 [3] (1,2,3) [**“Predicting Good Probabilities With Supervised Learning”**](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.60.7135&rep=rep1&type=pdf), A. Niculescu-Mizil & R. Caruana, ICML 2005

🔬 [4] (1,2) [**“Probabilistic Outputs for Support Vector Machines and Comparisons to Regularized Likelihood Methods”**](https://home.cs.colorado.edu/~mozer/Teaching/syllabi/6622/papers/Platt1999.pdf). J. Platt, (1999)

🔬 [5] [**“Transforming Classifier Scores into Accurate Multiclass Probability Estimates”**](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=04e10f745a7267453788a22f5150b5a32b2b3951). B. Zadrozny & C. Elkan, (KDD 2002)

🔬 [6] [**“Predicting accurate probabilities with a ranking loss”**](https://icml.cc/2012/papers/372.pdf). Menon AK, Jiang XJ, Vembu S, Elkan C, Ohno-Machado L. Proc Int Conf Mach Learn. 2012;2012:703-710

🔬 [7] [**“Beyond sigmoids: How to obtain well-calibrated probabilities from binary classifiers with beta calibration”**](https://projecteuclid.org/journals/electronic-journal-of-statistics/volume-11/issue-2/Beyond-sigmoids--How-to-obtain-well-calibrated-probabilities-from/10.1214/17-EJS1338SI.pdf) Kull, M., Silva Filho, T. M., & Flach, P. (2017).

📚 [8] Mario V. Wüthrich, Michael Merz (2023). [**“Statistical Foundations of Actuarial Learning and its Applications”**](https://link.springer.com/book/10.1007/978-3-031-12409-9) Springer Actuarial